In [1]:
import numpy as np
from stdb import load_db 
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import glob, os

import warnings  
from obspy.geodetics.base import gps2dist_azimuth
from obspy.core.event.catalog import read_events
warnings.filterwarnings("ignore")  
from  PlotCatalog_1 import plot_nonnull_average, plot_null_PCA09, plot_null_PCA_angle_small,plot_null_PCA_angle_large, plot_null_angle
                


In [2]:
PKLpath = '/Volumes/home/Research/STEP/01_Split'
Path = '/Volumes/home/Research/STEP/02_Station_result_csv'
# MAG = 6

SAVEpath = '/Volumes/home/Research/STEP/05_Station_result_statistics'
YearRange= 'SKSvsSKKS'
ver= 4

In [23]:
iiii=0
for STApath in glob.glob(f'{PKLpath}/*.pkl'):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]
    NET = values['network']; STA = values['station']
    if STA == 'CMCY' or STA == 'DGRL' or NET == 'BI' :
        pass
    else:
        newdf = pd.DataFrame()
        for csvpath in glob.glob(f'{Path}/*classify_220913/{NET}*{STA}*result_v2.csv'):
            phase = csvpath.rsplit('_')[4]
            df = pd.read_csv(csvpath)
            df = df[df['Pick']==True][df['Null']==False][df['Quality']=='Good']
            iiii += len(df)

In [24]:
iiii

51

In [27]:
allSTA = {'Station':[],
         'RCphi':[],'RCdphi':[],
         'RCdt':[],'RCddt':[],
         'SCphi':[],'SCdphi':[],
         'SCdt':[],'SCddt':[],
         'SKSnonNull':[], 'SKSNull':[],
         'SKKSnonNull':[], 'SKKSNull':[],
         'nonNull':[],'Null':[]}


for STApath in glob.glob(f'{PKLpath}/*.pkl'):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]

    NET = values['network']; STA = values['station']
    if STA == 'CMCY' or STA == 'DGRL' or NET == 'BI' :
        pass
    else:
        print(f'{NET}.{STA}')    
        if not os.path.isdir(f'{SAVEpath}/{NET}.{STA}'):
            os.mkdir(f'{SAVEpath}/{NET}.{STA}')
        allSTA['Station'].append(f'{NET}.{STA}')
        stlat = values['latitude']; stlon = values['longitude']

        newdf = pd.DataFrame()
        for csvpath in glob.glob(f'{Path}/*classify_220913/{NET}*{STA}*result_v2.csv'):
            phase = csvpath.rsplit('_')[4]
            df = pd.read_csv(csvpath)
            df = df[df['Pick']==True]
            newdf = newdf.append(df)
            if phase == 'SKS':
                if len(df) !=0:
                    allSTA['SKSnonNull'].append(len( df[df['Null']==False][df['Quality']!='Poor'] ))
                    allSTA['SKSNull'].append(len( df[df['Null']==True] ))
                else:
                    allSTA['SKSnonNull'].append(0)
                    allSTA['SKSNull'].append(0)  
            elif phase == 'SKKS':
                if len(df) !=0:
                    allSTA['SKKSnonNull'].append(len( df[df['Null']==False][df['Quality']!='Poor'] ))
                    allSTA['SKKSNull'].append(len( df[df['Null']==True] ))            
                else:
                    allSTA['SKKSnonNull'].append(0)
                    allSTA['SKKSNull'].append(0)             


        print(len(newdf))
        meanphiRC, stdphiRC, meandtRC, stddtRC, meanphiSC, stdphiSC, meandtSC, stddtSC, noNull, noNonNull = plot_nonnull_average(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)
        print(meanphiRC, stdphiRC, meandtRC, stddtRC, meanphiSC, stdphiSC, meandtSC, stddtSC, noNull, noNonNull)
        if np.isnan(meanphiRC) == False:
            allSTA['RCphi'].append(round(meanphiRC)); allSTA['RCdphi'].append(round(stdphiRC))
            allSTA['RCdt'].append(float(round(meandtRC,2))); allSTA['RCddt'].append( float(round(stddtRC,2)))
            allSTA['SCphi'].append(round(meanphiSC)); allSTA['SCdphi'].append(round(stdphiSC))
            allSTA['SCdt'].append(float(round(meandtSC,2))); allSTA['SCddt'].append( float(round(stddtSC,2)))
            allSTA['Null'].append(noNull); allSTA['nonNull'].append(noNonNull)
        else: 
            allSTA['RCphi'].append(meanphiRC); allSTA['RCdphi'].append(stdphiRC)
            allSTA['RCdt'].append(meandtRC); allSTA['RCddt'].append(stddtRC)
            allSTA['SCphi'].append(meanphiSC); allSTA['SCdphi'].append(stdphiSC)
            allSTA['SCdt'].append(meandtSC); allSTA['SCddt'].append(stddtSC)
            allSTA['Null'].append(noNull); allSTA['nonNull'].append(noNonNull)

#         plot_null_PCA09(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, meanphiSC, meandtSC, ver)
    #     plot_null_PCA_angle_small(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)
    #     plot_null_PCA_angle_large(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)  
    #     plot_null_angle(NET, STA, newdf, stlat, stlon, SAVEpath, YearRange, ver)  
    stadf = pd.DataFrame(allSTA)
    stadf.to_csv(f'{SAVEpath}/Station_SKS_{YearRange}average_v6.csv',index=False)


AM.ALAV
35
32.2737601935 25.8364245462 1.11050769203 0.550837745822 41.2741448415 16.9891713878 1.18081896806 0.463716867639 32 3
AM.ARZA
48
40.74 44.75 1.2 0.98 63.74 10.0 1.4 0.35 43 1
AM.BAUR
29
57.2725192305 16.8958204891 1.04717948725 0.349046319753 58.6735293112 11.5930714462 0.880018718382 0.280262773522 21 6
AM.BYUR
31
66.5120938885 31.6430284581 0.365664244985 0.692964645563 27.1473615317 12.6305680791 0.382906474376 0.383046994506 25 2
AM.GERK
32
nan nan nan nan nan nan nan nan 30 0
AM.KECH
8
nan nan nan nan nan nan nan nan 8 0
AM.LICH
26
nan nan nan nan nan nan nan nan 25 0
AM.MAGY
32
66.6087879421 24.308241915 0.947220183624 0.56189411814 68.7551787582 10.3274512829 0.729343228901 0.243823706805 30 2
AM.NAVR
19
39.0 44.75 0.4 0.98 54.0 44.75 0.5 0.98 16 1
AM.SHEN
26
43.71 44.75 1.2 0.98 27.71 44.75 1.3 0.98 21 1
AM.TSAP
14
-32.5761221034 31.6430284581 0.109737472316 0.692964645563 -22.905679458 23.8910286509 0.195173199271 0.517034815075 10 2
AM.VAND
22
43.2609484891 19.785

In [25]:
stadf = pd.DataFrame(allSTA)
stadf.to_csv(f'{SAVEpath}/Station_SK(K)S_2010-2020average_v{ver}.csv',index=False)

In [27]:
SAVEpath

'/Volumes/home/Research/STEP/05_Station_result_statistics'

In [6]:
1326+187

1513

In [28]:
f'{SAVEpath}/Station_SKS_{YearRange}average_v6.csv'

'/Volumes/home/Research/STEP/05_Station_result_statistics/Station_SKS_SKSvsSKKSaverage_v6.csv'